# Imports

In [1]:
import PCV
import wc1
import NecessarySets as ns
import PDACR9
import PDA9 as pda

# Variables

In [2]:
pcv_variables = {'string_separator': '\n', 'non_terminal_start': '<', 'non_terminal_finish': '>', 'arrow_tail': '-',
                 'arrow_head': '>', 'null_sequence_symbol': '%', 'empty_stack_symbol': '$', 'end_of_sequence_symbol': '!',
                 'non_terminal_pattern': '<[a-zA-Z]+>', 'arrow_op': '-{1}>{1}'}
pcv_variables['special_characters'] = [pcv_variables['arrow_tail'], pcv_variables['arrow_head'],
             pcv_variables['empty_stack_symbol'], pcv_variables['end_of_sequence_symbol'], pcv_variables['non_terminal_start'],
             pcv_variables['non_terminal_finish']]

ui_variables = {'w_title': 'Grammars', 'w_dimensions': '400x300', 'w_resizable': (False, False), 'txt_box_h': 25,
                'txt_box_w': 25, 'txt_box_coor': (0, 0), 'b_pda_name': 'Create Automata', 'c_b_name': 'Clear Grammar',
                'v_b_name': 'Validate String', 'b_width': 25, 'c_b_width': 25, 'e_width': 25, 'v_b_width': 25,
                'b_pda_coor': (225, 50), 'c_b_coor': (225, 100), 'e_coor': (225, 175), 'v_b_coor': (225, 225),
                'input_err_m': 'Your input does not follow the convention', 'err_0': 'Your productions are not well written',
                'err_1': 'There is an infinite cycle in your productions', 'err_2': 'The string must end with !',
                'err_3': 'These characters are reserved: ' + ''.join(pcv_variables['special_characters']),
                'err_4': 'You have not created an automata yet', 'err_5': 'This field cannot be empty',
                'err_6': 'The character !, goes at the end', 'label_1_txt': 'Enter a string to validate',
                'label_1_coor': (225, 150)}

# Application

In [3]:
class App:
    
    def __init__(self, pcv_variables, ui_variables):
        self.pcv_variables = pcv_variables
        self.ui_variables = ui_variables
        self.window = None
        self.txt_box = None
        self.b_pda = None
        self.clear_button = None
        self.production_creator = PCV.ProductionCreator(pcv_variables['string_separator'], pcv_variables['non_terminal_start'],
                                                       pcv_variables['non_terminal_finish'], pcv_variables['arrow_tail'],
                                                       pcv_variables['arrow_op'])
        self.production_validator = None
        self.production_set = None
        self.pda_creator = None
        self.push_down_automata = None
        self.entry = None
        self.validation_button = None
        self.string_to_validate = ''
        self.grammar = ''
        self.label_1 = None
        
    def start(self):
        self.window = wc1.create_main_window(self.ui_variables['w_title'], self.ui_variables['w_dimensions'],
                                       self.ui_variables['w_resizable'])
        self.txt_box = wc1.create_text_box(self.window, self.ui_variables['txt_box_h'], self.ui_variables['txt_box_w'])
        self.txt_box.place(x=self.ui_variables['txt_box_coor'][0], y=self.ui_variables['txt_box_coor'][1])
        self.b_pda = wc1.create_button(self.window, self.ui_variables['b_pda_name'], self.ui_variables['b_width'],
                                      self.create_push_down_automata)
        self.b_pda.place(x=self.ui_variables['b_pda_coor'][0], y=self.ui_variables['b_pda_coor'][1])
        self.clear_button = wc1.create_button(self.window, self.ui_variables['c_b_name'], self.ui_variables['c_b_width'],
                                      self.clear_grammars)
        self.clear_button.place(x=self.ui_variables['c_b_coor'][0], y=self.ui_variables['c_b_coor'][1])
        self.entry = wc1.create_entry(self.window, self.string_to_validate, self.ui_variables['e_width'])
        self.entry.place(x=self.ui_variables['e_coor'][0], y=self.ui_variables['e_coor'][1])
        self.validation_button = wc1.create_button(self.window, self.ui_variables['v_b_name'], self.ui_variables['v_b_width'],
                                      self.validate_string)
        self.validation_button.place(x=self.ui_variables['v_b_coor'][0], y=self.ui_variables['v_b_coor'][1])
        self.label_1 = wc1.create_label(self.window, self.ui_variables['label_1_txt'])
        self.label_1.place(x=self.ui_variables['label_1_coor'][0], y=self.ui_variables['label_1_coor'][1])
        self.window.mainloop()
        
    def valid_productions(self, productions):
        valid = True
        if not self.production_validator.productions_are_well_written(productions):
            valid = False
            wc1.show_error_message(self.ui_variables['err_0'])
        if self.production_validator.production_with_infinite_cycle(productions):
            valid = False
            wc1.show_error_message(self.ui_variables['err_1'])
        return valid
    
    def get_selection_set(self):
        nullable_non_terminals = self.production_set.get_nullable_non_terminals()
        print(nullable_non_terminals)
        nullable_productions = self.production_set.get_nullable_productions(nullable_non_terminals)
        set_of_firsts_for_non_terminals = self.production_set.replace_non_terminals(self.production_set.get_firsts_base_cases(
            nullable_non_terminals))
        set_of_firsts_for_productions = self.production_set.get_set_of_firsts_for_productions(nullable_non_terminals,
                                                                                set_of_firsts_for_non_terminals)
        set_of_nexts = self.production_set.replace_non_terminals(self.production_set.get_nexts_base_cases(
            nullable_non_terminals, set_of_firsts_for_non_terminals))
        selection_set = self.production_set.get_set_of_selection_for_productions(nullable_productions,
                                                                   set_of_firsts_for_productions, set_of_nexts)
        return selection_set
    
    def convert_set_to_string(self, _set):
        string = '{'
        for element in _set:
            if not _set.index(element) == len(_set)-1:
                string += (element + ', ')
            else:
                string += (element + '}')
        return string
            
    def obtain_pda(self):
        results = self.pda_creator.create_PDA()
        report = ''
        for result in results:
            if result[2] == 0:
                report += (result[0] +  '\n'+ '\n')
                if not self.push_down_automata and result[1]:
                    self.push_down_automata = result[1]
            elif result[2] == 1:
                report += (result[0] + '\n'+ '\n')
                for production in result[1]:
                    if not result[1].index(production) == len(result[1])-1:
                        report += (production.left_side + '->' + ''.join(production.right_side) + ', ')
                    else:
                        report += (production.left_side + '->' +''.join(production.right_side) + '\n'+ '\n')
            elif result[2] == 2 or result[2] == 3:
                report += (result[0] +  '\n'+ '\n')
                production = result[1]
                report += (production.left_side + '->' +''.join(production.right_side) + '\n'+ '\n')
            elif result[2] == 4:
                report += (result[0] + '\n'+ '\n')
                for pairs in result[1]:
                    production = pairs[0]
                    sets = list(pairs[1])
                    sets = self.convert_set_to_string(sets)
                    if not result[1].index(pairs) == len(result[1])-1:
                        report += ('production: ' + production.left_side + '->' + ''.join(production.right_side) +
                                   ' with selection set: '+ sets + '\n')
                    else:
                        report += ('production: ' + production.left_side + '->' + ''.join(production.right_side) +
                                   ' with selection set: '+ sets + '\n'+ '\n')                
        wc1.show_info_message(report)
        if self.push_down_automata:
            wc1.show_info_message('Automata Created')
            self.grammar = self.txt_box.get("1.0","end-1c")
        else:
            wc1.show_info_message('Automata was not Created')
    
    def clear_grammars(self):
        self.push_down_automata = None
        self.txt_box.delete("1.0","end-1c")
        self.grammar = ''
    
    def string_is_well_written(self, string):
        well_written = True
        try:
            if not string.index(self.pcv_variables['end_of_sequence_symbol']) == len(string)-1:
                wc1.show_error_message(self.ui_variables['err_6'])
                well_written = False
            else:
                for symbol in string:
                    if (symbol in self.pcv_variables['special_characters'] and
                        symbol != self.pcv_variables['end_of_sequence_symbol']):
                        wc1.show_error_message(self.ui_variables['err_3'])
                        well_written = False
                        break
        except:
            well_written = False
            wc1.show_error_message(self.ui_variables['err_2'])
        return  well_written
    
    def convert_string_to_list(self, string):
        _list = []
        for symbol in string:
            _list.append(symbol)
        return _list
    
    def validate_string(self):
        if self.push_down_automata:
            string = self.entry.get()
            if string:
                if self.string_is_well_written(string):
                    string_list = self.convert_string_to_list(string)
                    if self.push_down_automata.belongs_to_language(string_list):
                        wc1.show_info_message('The string: ' +  string + ' '+'belongs to the language generated by the grammar:' +
                                             '\n' + self.grammar)
                    else:
                        wc1.show_info_message('The string: ' +  string + ' '+
                                              'does not belong to the language generated by the grammar:' +
                                             '\n' + self.grammar)
            else:
                wc1.show_error_message(self.ui_variables['err_5'])
        else:
            wc1.show_error_message(self.ui_variables['err_4'])
        
    def create_push_down_automata(self):
        string = self.txt_box.get("1.0","end-1c")
        self.push_down_automata = None
        productions = self.production_creator.create_productions(self.production_creator.from_string_to_list(string))
        if not productions:
            wc1.show_error_message(self.ui_variables['input_err_m'])
        else:
            non_terminals = self.production_creator.get_non_terminals(productions)
            self.production_validator = PCV.ProductionValidator(non_terminals, self.pcv_variables['null_sequence_symbol'],
                                                               self.pcv_variables['non_terminal_pattern'],
                                                                self.pcv_variables['special_characters'])
            if self.valid_productions(productions):
                initial_non_terminal = non_terminals[0]
                self.production_set = ns.ProductionSet(initial_non_terminal, self.pcv_variables['null_sequence_symbol'],
                                                 self.pcv_variables['end_of_sequence_symbol'], productions, non_terminals)
                selection_set = self.get_selection_set()
                stack_operations = pda.StackOperation()
                self.pda_creator = PDACR9.PDACreator(self.pcv_variables['end_of_sequence_symbol'], 
                                                   self.pcv_variables['null_sequence_symbol'],
                                                   self.pcv_variables['empty_stack_symbol'],
                                                   initial_non_terminal, non_terminals, productions, stack_operations,
                                                   selection_set)
                self.obtain_pda()
            

# Start Application

In [4]:
app = App(pcv_variables, ui_variables)
app.start()

Exception in Tkinter callback
Traceback (most recent call last):
  File "C:\Users\Daniel\anaconda3\lib\tkinter\__init__.py", line 1705, in __call__
    return self.func(*args)
  File "<ipython-input-3-b1ecdb946aaf>", line 176, in create_push_down_automata
    selection_set = self.get_selection_set()
  File "<ipython-input-3-b1ecdb946aaf>", line 54, in get_selection_set
    nullable_non_terminals = self.production_set.get_nullable_non_terminals()
  File "C:\Users\Daniel\Desktop\grammars\NecessarySets.py", line 122, in get_nullable_non_terminals
    suspected_p = self.get_suspected_productions(nullable_non_terminals, no_nullable_non_terminals)
  File "C:\Users\Daniel\Desktop\grammars\NecessarySets.py", line 83, in get_suspected_productions
    if (self.only_right_side_of_non_terminals(production) and not production.left_side in nullable_non_terminals and
  File "C:\Users\Daniel\Desktop\grammars\NecessarySets.py", line 42, in only_right_side_of_non_terminals
    if not self.is_non_termina